In [12]:
import pandas as pd
from dfply import *

## Data In / Data Out

In [2]:
dat = pd.read_csv("gapminder.csv")

In [3]:
dat.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [6]:
# Customization 
pd.read_csv("gapminder.csv", 
            sep = ",", # Separator in the data
            index_col="country", # Set a variable to the index
            usecols = ["country","continent","year"], # Only request specific columns
            nrows = 3, # only read in n-rows of the data 
            na_values = "nan",
            parse_dates=True, # Parse all date features as datetime
            low_memory=True) # read the file in chunks for lower memory use (useful on large data)


,continent,year
country,,
Afghanistan,Asia,1952
Afghanistan,Asia,1957
Afghanistan,Asia,1962


In [7]:
# EXPORTING DATA

dat2 = dat.head()

# Export as csv without index
#dat2.to_csv("example.csv",index=False)

In [8]:
# DATA TYPE CONVERSIONS

# data ==> string
print(dat2.to_string())

# data ==> dict
dat2.to_dict()

# data ==> numpy array
dat2.values

# data ==> list
dat2.values.tolist()

       country continent  year  lifeExp       pop   gdpPercap
0  Afghanistan      Asia  1952   28.801   8425333  779.445314
1  Afghanistan      Asia  1957   30.332   9240934  820.853030
2  Afghanistan      Asia  1962   31.997  10267083  853.100710
3  Afghanistan      Asia  1967   34.020  11537966  836.197138
4  Afghanistan      Asia  1972   36.088  13079460  739.981106


[['Afghanistan', 'Asia', 1952, 28.801, 8425333, 779.4453145],
 ['Afghanistan', 'Asia', 1957, 30.332, 9240934, 820.8530296],
 ['Afghanistan', 'Asia', 1962, 31.997, 10267083, 853.10071],
 ['Afghanistan', 'Asia', 1967, 34.02, 11537966, 836.1971382],
 ['Afghanistan', 'Asia', 1972, 36.088, 13079460, 739.9811058]]

## Column Manipulation

In [10]:
dat2 = dat.sample(5, random_state=123)
dat2

,country,continent,year,lifeExp,pop,gdpPercap
1584,Uganda,Africa,1952,39.978,5824797,734.753484
249,Canada,Americas,1997,78.610,30305843,28954.925890
1142,Norway,Europe,1962,73.470,3638919,13450.401510
809,Jordan,Asia,1977,61.134,1937652,2852.351568
657,Honduras,Americas,1997,67.659,5867957,3160.454906


Dfply serves as the tidyverse-equivalent data manipulation library. Uses ">>" instead of ">%>" from R to pipe data manipulation functions together.

In [17]:
# each row, two listed columns
dat2.loc[:,['country','year']]

# pandas method for selecting columns
# same results without having to worry about specific slicing
dat2.filter(['country','year'])

# dfply version, X serves as the object being passed in
dat2 >> select(X.country, X.year)

,country,year
1584,Uganda,1952
249,Canada,1997
1142,Norway,1962
809,Jordan,1977
657,Honduras,1997


In [22]:
# each column between two columns
dat2.loc[:,"continent":"pop"]

#regular expression, anything that has a p in it
dat2.filter(regex="p")

# same as above without the need for the regex, easier to read
dat2 >> select(contains("p"))

,lifeExp,pop,gdpPercap
1584,39.978,5824797,734.753484
249,78.610,30305843,28954.925890
1142,73.470,3638919,13450.401510
809,61.134,1937652,2852.351568
657,67.659,5867957,3160.454906


In [23]:
#dropping variables
dat2.drop(columns=["year","lifeExp"])


# dfply approach
dat2 >> drop(X.year,X.lifeExp)

# negative select
dat2 >> select(~X.year,~X.lifeExp)

,country,continent,pop,gdpPercap
1584,Uganda,Africa,5824797,734.753484
249,Canada,Americas,30305843,28954.925890
1142,Norway,Europe,3638919,13450.401510
809,Jordan,Asia,1937652,2852.351568
657,Honduras,Americas,5867957,3160.454906


In [28]:
#dfply for rearranging columns

dat >> select(X.year, X.continent, X.country)

# functions the same as using .filter(), but has the added benefit of
# throwing a warning when a column is included twice, which the others don't

# to do so without dropping columns: (way easier than alternatives!)

dat2 >> select(X.year, X.country, everything())

,year,country,continent,lifeExp,pop,gdpPercap
1584,1952,Uganda,Africa,39.978,5824797,734.753484
249,1997,Canada,Americas,78.610,30305843,28954.925890
1142,1962,Norway,Europe,73.470,3638919,13450.401510
809,1977,Jordan,Asia,61.134,1937652,2852.351568
657,1997,Honduras,Americas,67.659,5867957,3160.454906


In [26]:
list(dat2) #gives us the equivalent of colnames() in R

['country', 'continent', 'year', 'lifeExp', 'pop', 'gdpPercap']

In [35]:
# Mutate equivalents

# in Pandas, reference new column name and assign value
dat2.loc[:, "lifeExp_new"] = dat2.lifeExp+100

# alternatively, assignment keeps it all in one function and allows us to use lambda functions as well
dat2 = dat2.assign(lifeExp_new2 = dat2.lifeExp/10)
dat2

# eval is great for working on larger datasets because its operating at the C level,
# wrap the entire function as a string because its evaluated after the python code has run.
# third-party or custom functions won't work here because of how it's compiled
dat2 = dat2.eval("lifeExp_new3 = sqrt(lifeExp)")
dat2

dat2 = dat2 >> mutate(lifeExp_new4 = X.lifeExp - X.lifeExp.mean())
dat2

,country,continent,year,lifeExp,pop,gdpPercap,lifeExp_new,lifeExp_new2,lifeExp_new3,lifeExp_new4
1584,Uganda,Africa,1952,39.978,5824797,734.753484,139.978,3.9978,6.322816,-24.1922
249,Canada,Americas,1997,78.610,30305843,28954.925890,178.610,7.8610,8.866228,14.4398
1142,Norway,Europe,1962,73.470,3638919,13450.401510,173.470,7.3470,8.571464,9.2998
809,Jordan,Asia,1977,61.134,1937652,2852.351568,161.134,6.1134,7.818823,-3.0362
657,Honduras,Americas,1997,67.659,5867957,3160.454906,167.659,6.7659,8.225509,3.4888


In [36]:
# need to provide the dictionary for original name : new name
dat2.rename(columns={"country":"country_name","lifeExp":"LE"})


# dfply approach, new name = X.oldname
dat2 >> rename(country_name = X.country, LE = X.lifeExp)

,country_name,continent,year,LE,pop,gdpPercap,lifeExp_new,lifeExp_new2,lifeExp_new3,lifeExp_new4
1584,Uganda,Africa,1952,39.978,5824797,734.753484,139.978,3.9978,6.322816,-24.1922
249,Canada,Americas,1997,78.610,30305843,28954.925890,178.610,7.8610,8.866228,14.4398
1142,Norway,Europe,1962,73.470,3638919,13450.401510,173.470,7.3470,8.571464,9.2998
809,Jordan,Asia,1977,61.134,1937652,2852.351568,161.134,6.1134,7.818823,-3.0362
657,Honduras,Americas,1997,67.659,5867957,3160.454906,167.659,6.7659,8.225509,3.4888


## Row-wise Manipulations

Filtering, Rearranging, Summarizing

In [40]:

# returns boolean value for each row
dat.lifeExp < 25

# returns the rows for which the boolean value is True
dat.loc[dat.lifeExp < 25]

# operates like eval, also wrapped in string
dat.query("lifeExp < 25")

# dfply for the above
dat >> mask(X.lifeExp < 25)

,country,continent,year,lifeExp,pop,gdpPercap
1292,Rwanda,Africa,1992,23.599,7290203,737.068595


In [44]:
dat.drop_duplicates("continent") # first values for each row are returned

# useful following a filter to get a summary list of factors
dat.filter(["continent"]).drop_duplicates("continent")

# same as first example in dfply
dat >> distinct(X.continent)

# same as second example in dfply
dat >> select(X.continent) >> distinct(X.continent)

,continent
0,Asia
12,Europe
24,Africa
48,Americas
60,Oceania


In [47]:
# slice rows with index in dfply
dat >> row_slice([200,201,202])

# sampling
dat.sample(3)

dat >> sample(3)

,country,continent,year,lifeExp,pop,gdpPercap
1430,Sri Lanka,Asia,1962,62.192,10421936,1074.471960
720,Iran,Asia,1952,44.869,17272000,3035.326002
1490,Syria,Asia,1962,50.305,4834621,2193.037133


In [49]:
dat.sort_values('country', ascending=False).head(3)

,country,continent,year,lifeExp,pop,gdpPercap
1703,Zimbabwe,Africa,2007,43.487,12311143,469.709298
1697,Zimbabwe,Africa,1977,57.674,6642107,685.587682
1692,Zimbabwe,Africa,1952,48.451,3080907,406.884115


In [51]:
dat >> arrange(X.country) >> head(3)

dat >> arrange(desc(X.country)) >> head(3)

# dfply: sort values by more than one column variable 
dat >> arrange(desc(X.country),X.year) >> head(3)

,country,continent,year,lifeExp,pop,gdpPercap
1692,Zimbabwe,Africa,1952,48.451,3080907,406.884115
1693,Zimbabwe,Africa,1957,50.469,3646340,518.764268
1694,Zimbabwe,Africa,1962,52.358,4277736,527.272182


In [52]:
# Built in aggregation functions
dat.lifeExp.mean()
dat.lifeExp.median()
dat.lifeExp.mode()
dat.lifeExp.sum()
dat.lifeExp.size


# using the .agg() method to summarize across variables
dat[['lifeExp','pop','gdpPercap']].agg(mean)


# dfply approach
# being able to give names to the summary varliables similar to R
dat >> summarize(lifeExp_mean = X.lifeExp.mean(),
                 lifeExp_std = X.lifeExp.std())

,lifeExp_mean,lifeExp_std
0,59.474439,12.917107


## Group Manipulations
Changing the unit of analysis.

Input -> Split -> Apply -> Combine -> Output

In [55]:
# 
g = dat.groupby(["continent"])
g # special groupby generator object

# In practice, we can chain together the group by
dat.groupby(["continent"]).head(2)   

# dfply: group by a column entries. 
dat >> group_by(X.continent) >> head(2)

# With dfply, the group_by() method will persist. 
# As we need to ungroup() if we wish to turn off the key.
d = dat >> group_by(X.continent) 
d >> head(2)
d >> ungroup() >> head(2)

('Africa',        country continent  year  lifeExp       pop    gdpPercap
24     Algeria    Africa  1952   43.077   9279525  2449.008185
25     Algeria    Africa  1957   45.685  10270856  3013.976023
26     Algeria    Africa  1962   48.303  11000948  2550.816880
27     Algeria    Africa  1967   51.407  12760499  3246.991771
28     Algeria    Africa  1972   54.518  14760787  4182.663766
...        ...       ...   ...      ...       ...          ...
1699  Zimbabwe    Africa  1987   62.351   9216418   706.157306
1700  Zimbabwe    Africa  1992   60.377  10704340   693.420786
1701  Zimbabwe    Africa  1997   46.809  11404948   792.449960
1702  Zimbabwe    Africa  2002   39.989  11926563   672.038623
1703  Zimbabwe    Africa  2007   43.487  12311143   469.709298

[624 rows x 6 columns])
('Americas',         country continent  year  lifeExp       pop     gdpPercap
48    Argentina  Americas  1952   62.485  17876956   5911.315053
49    Argentina  Americas  1957   64.399  19610538   6856.856212


In [61]:
dat.groupby(["continent"])[["lifeExp","gdpPercap"]].mean()
# group level average of specified columns

# group by continent, for two selected columns, perform each of the aggregate functions in the list argument
dat.groupby(["continent"])[['lifeExp','gdpPercap']].agg(["mean","std","median"])

# to be more specific, we can provide a dictionary to tell it specifically which methods to run on which columns
dat.groupby(['continent']).agg({"lifeExp":mean,"gdpPercap":median})

# Group by more than one variable --- resulting in a multi-index
dat.groupby(['continent','country'])[['lifeExp','gdpPercap']].mean()

lifeExp     gdpPercap
continent country                                
Africa    Algeria         59.030167   4426.025973
          Angola          37.883500   3607.100529
          Benin           48.779917   1155.395107
          Botswana        54.597500   5031.503557
          Burkina Faso    44.694000    843.990665
...                             ...           ...
Europe    Switzerland     75.565083  27074.334405
          Turkey          59.696417   4469.453380
          United Kingdom  73.922583  19380.472986
Oceania   Australia       74.662917  19980.595634
          New Zealand     73.989500  17262.622813

[142 rows x 2 columns]

In [65]:
# dfply versions of grouping
dat >> group_by(X.continent) >> summarize(lifeExp_mean = X.lifeExp.mean(),
            lifeExp_std = X.lifeExp.std())

# For summarization across a range of variables 
dat >> group_by(X.continent) >> summarize_each([np.mean,np.std],X.lifeExp,X.gdpPercap)

,continent,lifeExp_mean,lifeExp_std,gdpPercap_mean,gdpPercap_std
0,Africa,48.865330,9.142875,2193.754578,2825.662985
1,Americas,64.658737,9.329500,7136.110356,6386.093939
2,Asia,60.064903,11.849542,7902.150428,14027.627845
3,Europe,71.903686,5.425626,14469.475533,9342.211111
4,Oceania,74.326208,3.715695,18621.609223,6225.094993


In [66]:
# Pandas: groupby() + transform()
def center(x):
    '''Center a variable around its mean'''
    return x - x.mean()

dat.groupby('country')[["lifeExp","pop"]].transform(center).head(10)

# Likewise, apply() offers identical functionality. The only requirement of 
# apply is that the output must be a pandas.DataFrame, a pandas.Series, or a scalar.

# Pandas: groupby() + apply() using a custon function
dat.groupby('country')[["lifeExp","pop"]].apply(center).head(10)


# dfply: group_by + mutate(), also with custon functions
d = dat >> group_by(X.country) >> mutate(lifeExp_centered = center(X.lifeExp)) 
d.head(10)

# mutate multiple columns at once, same as with summarize
d = dat >> group_by(X.country) >> mutate(lifeExp_centered = center(X.lifeExp),le_ave = X.lifeExp.mean()) 
d.head(10)

# Return only the manipulated value
d = dat >> group_by(X.country) >> transmute(lifeExp_centered = center(X.lifeExp)) 
d.head(10)

,country,lifeExp_centered
0,Afghanistan,-8.677833
1,Afghanistan,-7.146833
2,Afghanistan,-5.481833
3,Afghanistan,-3.458833
4,Afghanistan,-1.390833
5,Afghanistan,0.959167
6,Afghanistan,2.375167
7,Afghanistan,3.343167
8,Afghanistan,4.195167
9,Afghanistan,4.284167
